In [1]:
import numpy as np
from scipy.integrate import quad
import pygame 

from formula_functions import *
from animation_functions import *
# from intersection_functions import create_spiral, distance
from show_parameters_functions import *
from key_controls_functions import *


# from variables import *
from Data_classes.data_processing import DataProcessing


pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
dd = DataProcessing()
dd.mode_statuses_dict

{'Coordinates': [<Surface(1500x800x32 SW)>, True],
 'Algorithm mode': [<Surface(1500x800x32 SW)>, False],
 'Algorithm data mode': [<Surface(1500x800x32 SW)>, True],
 'T-diagram': [None, False],
 'Vertical line': [<Surface(1500x800x32 SW)>, True],
 'Spiral': [<Surface(1500x800x32 SW)>, True],
 'Y-intersects': [<Surface(1500x800x32 SW)>, True],
 'Line-intersects': [<Surface(1500x800x32 SW)>, True],
 'Parameters': [<Surface(1500x800x32 SW)>, True],
 'Modes layer': [<Surface(1500x800x32 SW)>, True],
 'Derivatives': [<Surface(1500x800x32 SW)>, False]}

In [3]:
def main_loop(screen_width, screen_height, FPS):

    '''
    Initializations
    
    '''
    
    pygame.init()

    font_small = pygame.font.Font(None, 26)

    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer

    data_processing = DataProcessing()
    # data_processing.reset_dicts()

    t_mth_aproxim_list = [] # caches the already computed approximation values
    
    
    # Create background coordinate system
    create_background(data_processing, font_small)

    draw_vertical_line(data_processing)
    
    
    draw_derivatives(data_processing)

    draw_spiral(data_processing)
    
    t_nth_list = calc_y_intersects_t(data_processing)
   

        
    # Initial drawing of the y-intersection points
    draw_dots(data_processing, t_nth_list, 'Y-intersects')

    show_parameters(data_processing, font_small)
    
    show_mode_statuses(data_processing, font_small)
    
    
    data_processing.blit_layers(win)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(data_processing.constants.FPS)

        # All updates booleans must be set to False
        data_processing.booleans.reset_to_false()
      
        # Check and handle for key control events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
    
                handle_ctrl_commands(event, data_processing)
                handle_switch_commands(event, data_processing)
    
                if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                    t_mth_aproxim_list = handle_algorithm_mode_controls(event, data_processing, t_mth_aproxim_list)
                    
                else:

                    handle_shift_key_commands(event, data_processing)

                if data_processing.booleans.update_booleans_dict['is_turn_off'] is not None:
                    
                    data_processing.mode_statuses_dict['Algorithm mode'][0].fill((0, 0, 0,0))
                    data_processing.mode_statuses_dict['Algorithm data mode'][0].fill((0, 0, 0, 0))
                    t_mth_aproxim_list.clear()
                    data_processing.algorithm_vars.reset_dict()
                    data_processing.mode_statuses_dict['Derivatives'][1] = False

        handle_key_commands(data_processing)

        if data_processing.booleans.update_booleans_dict['update_screen']:

            if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                
                data_processing.mode_statuses_dict['Derivatives'][1] = False
                
            draw_derivatives(data_processing)
            
            if data_processing.booleans.update_booleans_dict['shift_coords'] \
                or data_processing.booleans.update_booleans_dict['reset_background']:
               
                # Update coordinate system
                create_background(data_processing, font_small)

            
            if data_processing.booleans.update_booleans_dict['update_spiral'] \
                or data_processing.booleans.update_booleans_dict['is_t_diagram_change']:
            
                draw_spiral(data_processing)
    
            if data_processing.booleans.update_booleans_dict['update_line']:         
                
                draw_vertical_line(data_processing)
             

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(data_processing)
            
            # Draw current y intersection points if y-intersects mode is `on`
            if data_processing.mode_statuses_dict['Y-intersects'][1]:
             
                draw_dots(data_processing, t_nth_list, 'Y-intersects')

            # A list contains the exact t-parameters for the intersection points of the spiral with the vertical line
            t_mth_list = calc_line_intersections_t(data_processing, t_nth_list)
            
            # Draw current line intersection points if line-intersects mode is `on`
            if data_processing.mode_statuses_dict['Line-intersects'][1]:
                draw_dots(data_processing, t_mth_list, 'Line-intersects')
                
            # Show the current statuses of the spiral and line parameters
            show_parameters(data_processing, font_small)  
            
            # Show the current statuses of the mode parameters
            show_mode_statuses(data_processing, font_small)

            
            
            # Show the algorithm steps towards the interesection point
            if data_processing.mode_statuses_dict['Algorithm mode'][1]:

                t_mth_aproxim_list = draw_algorithm_steps(data_processing, t_nth_list, t_mth_aproxim_list, accuracy= 5)
            
                x, y = 'None', 'None'
                if len(t_mth_aproxim_list) > 1:
            
                    mth_t = t_mth_aproxim_list[data_processing.algorithm_vars.algorithm_vars_dict['m']]
            
                    x, y = calc_single_t_aproxim(data_processing, mth_t, transform=False)
                
                # Show the current statuses of the m_th aproximation point
                show_algorithm_rows_and_cols(data_processing, x, y, font_small)
                
            # Attache the new drawings on the screen    
            data_processing.blit_layers(win)
            
            pygame.display.update()

        
    # Exit
    pygame.quit()
    
main_loop(screen_width, screen_height, FPS)